In [36]:
import pandas as pd

project_path = '/home/wjunneng/Ubuntu/2019-CCF-Passenger-Car-Segment-Sales-Forecast'

# train_sales_data
train_sales_data_path = project_path + '/data/original/train_sales_data.csv'
# train_search_data
train_search_data_path = project_path + '/data/original/train_search_data.csv'
# train_user_reply_data
train_user_reply_data = project_path + '/data/original/train_user_reply_data.csv'

# evaluation_public
evaluation_public = project_path + '/data/original/evaluation_public.csv'
# submit_example.csv
submit_example = project_path + '/data/original/submit_example.csv'

train_sales_data = pd.read_csv(train_sales_data_path)
train_search_data = pd.read_csv(train_search_data_path)
train_user_reply_data = pd.read_csv(train_user_reply_data)
evaluation_public = pd.read_csv(evaluation_public)



In [37]:
print(train_search_data.info())
print('\n')
print(train_sales_data.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31680 entries, 0 to 31679
Data columns (total 6 columns):
province      31680 non-null object
adcode        31680 non-null int64
model         31680 non-null object
regYear       31680 non-null int64
regMonth      31680 non-null int64
popularity    31680 non-null int64
dtypes: int64(4), object(2)
memory usage: 1.5+ MB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31680 entries, 0 to 31679
Data columns (total 7 columns):
province       31680 non-null object
adcode         31680 non-null int64
model          31680 non-null object
bodyType       31680 non-null object
regYear        31680 non-null int64
regMonth       31680 non-null int64
salesVolume    31680 non-null int64
dtypes: int64(4), object(3)
memory usage: 1.7+ MB
None


In [38]:
data = pd.concat([train_sales_data, evaluation_public], axis=0, ignore_index=True)

data = data.merge(train_search_data, how='left', on=['province', 'adcode', 'model'
                                                                            , 'regYear', 'regMonth'])
print(data.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 36960 entries, 0 to 36959
Data columns (total 10 columns):
adcode           36960 non-null int64
bodyType         31680 non-null object
forecastVolum    0 non-null float64
id               5280 non-null float64
model            36960 non-null object
province         36960 non-null object
regMonth         36960 non-null int64
regYear          36960 non-null int64
salesVolume      31680 non-null float64
popularity       31680 non-null float64
dtypes: float64(4), int64(3), object(3)
memory usage: 3.1+ MB
None


/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [39]:
data = data.merge(train_user_reply_data, how='left', on=['model', 'regYear', 'regMonth'])

print(data.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 36960 entries, 0 to 36959
Data columns (total 12 columns):
adcode             36960 non-null int64
bodyType           31680 non-null object
forecastVolum      0 non-null float64
id                 5280 non-null float64
model              36960 non-null object
province           36960 non-null object
regMonth           36960 non-null int64
regYear            36960 non-null int64
salesVolume        31680 non-null float64
popularity         31680 non-null float64
carCommentVolum    31680 non-null float64
newsReplyVolum     31680 non-null float64
dtypes: float64(6), int64(3), object(3)
memory usage: 3.7+ MB
None


In [40]:
data['id'] = data['id'].fillna(0).astype(int)
del data['forecastVolum']

print(data.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 36960 entries, 0 to 36959
Data columns (total 11 columns):
adcode             36960 non-null int64
bodyType           31680 non-null object
id                 36960 non-null int64
model              36960 non-null object
province           36960 non-null object
regMonth           36960 non-null int64
regYear            36960 non-null int64
salesVolume        31680 non-null float64
popularity         31680 non-null float64
carCommentVolum    31680 non-null float64
newsReplyVolum     31680 non-null float64
dtypes: float64(4), int64(4), object(3)
memory usage: 3.4+ MB
None


In [41]:
data['bodyType'] = data['model'].map(train_sales_data.drop_duplicates('model').set_index('model')['bodyType'])
print(data.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 36960 entries, 0 to 36959
Data columns (total 11 columns):
adcode             36960 non-null int64
bodyType           36960 non-null object
id                 36960 non-null int64
model              36960 non-null object
province           36960 non-null object
regMonth           36960 non-null int64
regYear            36960 non-null int64
salesVolume        31680 non-null float64
popularity         31680 non-null float64
carCommentVolum    31680 non-null float64
newsReplyVolum     31680 non-null float64
dtypes: float64(4), int64(4), object(3)
memory usage: 3.4+ MB
None


In [42]:
data.head()



,adcode,bodyType,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,292.0,1479.0,11.0,106.0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,466.0,1594.0,11.0,106.0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,257.0,1479.0,11.0,106.0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,408.0,2370.0,11.0,106.0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,610.0,3562.0,11.0,106.0


In [43]:
for column in ['bodyType', 'model']:
    data[column] = data[column].map(dict(zip(data[column].unique(), range(data[column].nunique()))))

data.head()



,adcode,bodyType,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum
0,310000,0,0,0,上海,1,2016,292.0,1479.0,11.0,106.0
1,530000,0,0,0,云南,1,2016,466.0,1594.0,11.0,106.0
2,150000,0,0,0,内蒙古,1,2016,257.0,1479.0,11.0,106.0
3,110000,0,0,0,北京,1,2016,408.0,2370.0,11.0,106.0
4,510000,0,0,0,四川,1,2016,610.0,3562.0,11.0,106.0


In [47]:
data['month'] = (data['regYear'] - 2016) * 12 + data['regMonth']
shift_feat = []

data['model_adcode'] = data['adcode'] + data['model']
data['model_adcode_month'] = data['model_adcode'] * 100 + data['month']
for i in [11]:
    i = i + 1
    shift_feat.append('shift_model_adcode_month_salesVolume_{0}'.format(i))
    data['model_adcode_month_{0}'.format(i)] = data['model_adcode_month'] + i
    data_last = data[~data.salesVolume.isnull()].set_index('model_adcode_month_{0}'.format(i))
    data['shift_model_adcode_month_salesVolume_{0}'.format(i)] = data['model_adcode_month'].map(data_last['salesVolume'])
print(shift_feat)
print(data.info())



['shift_model_adcode_month_salesVolume_12']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 36960 entries, 0 to 36959
Data columns (total 16 columns):
adcode                                     36960 non-null int64
bodyType                                   36960 non-null int64
id                                         36960 non-null int64
model                                      36960 non-null int64
province                                   36960 non-null object
regMonth                                   36960 non-null int64
regYear                                    36960 non-null int64
salesVolume                                31680 non-null float64
popularity                                 31680 non-null float64
carCommentVolum                            31680 non-null float64
newsReplyVolum                             31680 non-null float64
month                                      36960 non-null int64
model_adcode                               36960 non-null int64
model_

In [48]:
data.head()



,adcode,bodyType,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum,month,model_adcode,model_adcode_month,model_adcode_month_12,shift_model_adcode_month_salesVolume_12
0,310000,0,0,0,上海,1,2016,292.0,1479.0,11.0,106.0,1,310000,31000001,31000013,NaN
1,530000,0,0,0,云南,1,2016,466.0,1594.0,11.0,106.0,1,530000,53000001,53000013,NaN
2,150000,0,0,0,内蒙古,1,2016,257.0,1479.0,11.0,106.0,1,150000,15000001,15000013,NaN
3,110000,0,0,0,北京,1,2016,408.0,2370.0,11.0,106.0,1,110000,11000001,11000013,NaN
4,510000,0,0,0,四川,1,2016,610.0,3562.0,11.0,106.0,1,510000,51000001,51000013,NaN


In [49]:
print(data.columns)



Index(['adcode', 'bodyType', 'id', 'model', 'province', 'regMonth', 'regYear',
       'salesVolume', 'popularity', 'carCommentVolum', 'newsReplyVolum',
       'month', 'model_adcode', 'model_adcode_month', 'model_adcode_month_12',
       'shift_model_adcode_month_salesVolume_12'],
      dtype='object')


In [ ]:
num_feat = ['regYear'] + shift_feat
cate_feat = ['adcode', 'bodyType', 'model', 'regMonth']

features = num_feat + cate_feat

# data['n_label'] = data['label'] / data.groupby('model')['label'].transform('mean')
train_idx = (data['mt'] <= 20)

valid_idx = (data['mt'].between(21, 24))

test_idx = (data['mt'] > 24)

data['model_weight'] = data.groupby('model')['label'].transform('mean')
data['n_label'] = data['label'] / data['model_weight']

train_x = data[train_idx][features]
train_y = data[train_idx]['n_label']

valid_x = data[valid_idx][features]
valid_y = data[valid_idx]['n_label']
